In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

data_dir = '/home/maith/Desktop/cityscapes'
train_images_dir = os.path.join(data_dir, 'leftImg8bit/train')
train_labels_dir = os.path.join(data_dir, 'gtFine/train')

def preprocess_image(image_path, label_path, target_size=(256, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label)
    return image, tf.cast(label, tf.int32)

def file_paths_generator():
    count = 0
    for city in sorted(os.listdir(train_images_dir)):
        city_images_path = os.path.join(train_images_dir, city)
        city_labels_path = os.path.join(train_labels_dir, city)
        for image_name in sorted(os.listdir(city_images_path)):
            if image_name.endswith('_leftImg8bit.png'):
                image_path = os.path.join(city_images_path, image_name)
                label_name = image_name.replace('_leftImg8bit.png', '_gtFine_labelIds.png')
                label_path = os.path.join(city_labels_path, label_name)
                count += 1
                yield image_path, label_path
    print(f'Total images processed: {count}')

def create_dataset():
    dataset = tf.data.Dataset.from_generator(
        generator=file_paths_generator,
        output_types=(tf.string, tf.string),
        output_shapes=((), ()))

    dataset = dataset.map(lambda x, y: preprocess_image(x, y))
    return dataset.repeat()

dataset = create_dataset().batch(8).prefetch(buffer_size=tf.data.AUTOTUNE)

try:
    for batch, (images, labels) in enumerate(dataset.take(2)):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f'Sample Image from Batch {batch+1}')
        plt.imshow(images[0].numpy().astype('uint8'))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f'Sample Label from Batch {batch+1}')
        plt.imshow(labels[0].numpy().squeeze(), cmap='gray')
        plt.axis('off')
        plt.show()
except tf.errors.OutOfRangeError:
    print("Attempted to access beyond the available data.")

In [ ]:
import os
import numpy as np
import tensorflow as tf

data_dir = '/home/maith/Desktop/cityscapes'
train_labels_dir = os.path.join(data_dir, 'gtFine/train')

def analyze_dataset(train_labels_dir):
    class_ids = []
    for city in sorted(os.listdir(train_labels_dir)):
        city_labels_path = os.path.join(train_labels_dir, city)
        for label_file in sorted(os.listdir(city_labels_path)):
            if label_file.endswith('_labelIds.png'):
                label_path = os.path.join(city_labels_path, label_file)
                label = tf.io.read_file(label_path)
                label = tf.image.decode_png(label, channels=1)
                unique_ids = np.unique(label.numpy())
                class_ids.extend(unique_ids)
    
    unique_class_ids = np.unique(class_ids)
    return unique_class_ids

# Get unique class IDs
unique_class_ids = analyze_dataset(train_labels_dir)
print(f"Unique class IDs in the dataset: {unique_class_ids}")
print(f"Total number of classes: {len(unique_class_ids)}")

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def unet_model(input_shape=(256, 512, 3), num_classes=34):
    inputs = Input(input_shape)
    
    # Downsample
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

    # Upsample
    u1 = UpSampling2D((2, 2))(b)
    concat1 = concatenate([u1, c2])
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)
    u2 = UpSampling2D((2, 2))(c3)
    concat2 = concatenate([u2, c1])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)

    # Output
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c4)

    model = Model(inputs=inputs, outputs=outputs)
    return model

model = unet_model()
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import MeanIoU

def compile_model(model):
    optimizer = Adam(learning_rate=0.001)
    loss = SparseCategoricalCrossentropy()
    metrics = ['accuracy', MeanIoU(num_classes=34)]

    model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    )

compile_model(model)

print("Model compiled successfully with Adam optimizer, Sparse Categorical Crossentropy loss, and accuracy & MeanIoU metrics.")

In [ ]:
import tensorflow as tf

def load_and_preprocess_image(image_path, label_path, target_size=(256, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label, axis=-1)

    return image, label

def prepare_dataset(image_dir, label_dir, batch_size=8):
    image_paths = [os.path.join(image_dir, city, f) for city in os.listdir(image_dir) for f in os.listdir(os.path.join(image_dir, city)) if f.endswith('_leftImg8bit.png')]
    label_paths = [p.replace('_leftImg8bit.png', '_gtFine_labelIds.png').replace('leftImg8bit', 'gtFine') for p in image_paths]

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

train_image_dir = os.path.join(data_dir, 'leftImg8bit/train')
train_label_dir = os.path.join(data_dir, 'gtFine/train')
val_image_dir = os.path.join(data_dir, 'leftImg8bit/val')
val_label_dir = os.path.join(data_dir, 'gtFine/val')

train_dataset = prepare_dataset(train_image_dir, train_label_dir)
val_dataset = prepare_dataset(val_image_dir, val_label_dir)

print("Training and validation datasets are ready.")

In [ ]:
def check_dataset_shapes(dataset):
    for images, labels in dataset.take(1):
        print("Images shape:", images.shape)
        print("Labels shape:", labels.shape)

check_dataset_shapes(train_dataset)
check_dataset_shapes(val_dataset)

In [ ]:
from tensorflow.keras.layers import Conv2D

def check_model_output(model, input_shape=(1, 256, 512, 3)):
    test_input = tf.random.normal(input_shape)
    test_output = model(test_input)
    print("Test output shape:", test_output.shape)

check_model_output(model)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_callback = ModelCheckpoint(
    os.path.join('/home/maith/Desktop/cityscapes/', 'best_model.keras'), 
    monitor='val_loss', 
    save_best_only=True, 
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, early_stopping_callback],
    verbose=1
)

print("Training complete.")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


In [ ]:
import numpy as np

true_labels = []
pred_labels = []

for images, labels in val_dataset:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=-1)
    true_labels.extend(labels.numpy().flatten())
    pred_labels.extend(preds.flatten())

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=unique_class_ids, yticklabels=unique_class_ids)
plt.title('Normalized Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np

def representative_dataset_gen():
    dataset = create_dataset().batch(1)
    for image, _ in dataset.take(100):
        yield [image]

model = tf.keras.models.load_model('final_model.keras')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

try:
    tflite_model = converter.convert()
    print("Model converted successfully.")
except Exception as e:
    print(f"An error occurred during conversion: {str(e)}")
    raise

with open('model_fully_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model has been successfully converted, fully quantized, and saved.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

def convert_to_tflite(model_path, tflite_model_path, val_dataset):
    model = load_model(model_path)

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

    def representative_data_gen():
        for input_value, _ in val_dataset.take(10):
            yield [input_value]

    converter.representative_dataset = representative_data_gen
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8

    tflite_model = converter.convert()

    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model saved as {tflite_model_path}")

# Path to your saved Keras model
model_path = 'final_model.keras'

# Desired path for the TFLite model
tflite_model_path = 'model_quantized.tflite'

# Assuming you have a `val_dataset` for the representative dataset
convert_to_tflite(model_path, tflite_model_path, val_dataset)

In [ ]:
import tensorflow as tf

# Load the TFLite model
tflite_model_path = 'model_quantized_no_tile.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get the list of all operations in the model
ops = interpreter._get_ops_details()
op_names = [op['op_name'] for op in ops]
print(op_names)

In [ ]:
def test_model_segment(model_segment):
    converter = tf.lite.TFLiteConverter.from_keras_model(model_segment)
    tflite_model = converter.convert()

    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    ops_details = interpreter._get_ops_details()
    op_names = [op['op_name'] for op in ops_details]
    if 'TILE' in op_names:
        print("TILE operation found in this segment!")
    print("Operations in this segment:", op_names)

# Usage: create a model segment and pass it to the function
segment_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(256, 512, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    # Add more layers incrementally
])
test_model_segment(segment_model)

In [7]:
import tensorflow as tf

# Load your final Keras model
model = tf.keras.models.load_model('/home/maith/Desktop/cityscapes/best_model.h5')

print("TensorFlow version:", tf.__version__)

# Convert the model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set the converter to use TFLITE_BUILTINS only
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

converter.experimental_new_converter = False
print("Successfully set experimental_new_converter to False")

tflite_model = converter.convert()
print("Conversion successful!")

# Save the TensorFlow Lite model
with open('model_static.tflite', 'wb') as f:
    f.write(tflite_model)

2024-07-20 10:33:05.559967: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-20 10:33:05.565706: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-20 10:33:05.565743: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ubuntu
2024-07-20 10:33:05.565756: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ubuntu
2024-07-20 10:33:05.565852: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.183.1
2024-07-20 10:33:05.565890: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.183.1
2024-07-20 10:33:05.565902: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.183.1
2024-07-20 10:33:05.566349: I tensorflow/core/platform/cpu_feature_guard.cc:142] This Tens

TensorFlow version: 2.5.0
Successfully set experimental_new_converter to False


2024-07-20 10:33:12.865049: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/home/maith/.pyenv/versions/3.7.12/envs/csenv/lib/python3.7/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmpc8k7q1k9/assets


2024-07-20 10:33:19.590533: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-20 10:33:19.590677: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-07-20 10:33:19.592190: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2419200000 Hz
2024-07-20 10:33:19.601066: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.

2024-07-20 10:33:20.180482: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-20 10:33:20.180576: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-07-20 10:33:20.311126: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimizat

Conversion successful!


In [ ]:
import tensorflow as tf

def test_model_segment(layers):
    input_tensor = tf.keras.Input(shape=(256, 512, 3))
    x = input_tensor
    for layer in layers:
        x = layer(x)
    model = tf.keras.Model(inputs=input_tensor, outputs=x)

    # Convert the model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Check operations
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
    ops_set = set(detail['op_name'] for detail in interpreter._get_ops_details())
    print("Operations:", ops_set)

# Example: Incrementally add layers and test
layers_to_test = [
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
]
test_model_segment(layers_to_test)